In [ ]:
!pip install prf-api

     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
     |████████████████████████████████| 112kB 46.6MB/s 
  Created wheel for unrar: filename=unrar-0.3-cp36-none-any.whl size=21339 sha256=f48ac9e6b544f9a18fcb78d6a1b11f2fc414be952d23a1df7e322a2c3040fb88
  Stored in directory: /root/.cache/pip/wheels/c7/a9/1a/c5a381df313452ec2a000c9eae2188a2963df5f2f940073a5f
  Created wheel for rarfile: filename=rarfile-3.0-cp36-none-any.whl size=24207 sha256=065debad00b9ff6a26717e3f3b429ed7c76a121f3d77187b9326a85fd13c4fd5
  Stored in directory: /root/.cache/pip/wheels/dc/84/da/8aff50941f548db5384b076d5a6a6afea0cd12672e0326edc4
Successfully built unrar rarfile
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.21.0
    Uninstalling re

In [ ]:
from prf_api.prf_api import PRFApi
import pandas as pd

import re

In [ ]:
data = PRFApi()
accidents_df = data.dataframe('acidentes_ocorrencia', estado='RN')

Buscando datasets de acidentes_ocorrencia para o ano 2019...
>> Baixando 2019/datatran2019.csv
Buscando datasets de acidentes_ocorrencia para o ano 2018...
>> Baixando 2018/datatran2018.csv
Buscando datasets de acidentes_ocorrencia para o ano 2017...
>> Baixando 2017/datatran2017.csv
Buscando datasets de acidentes_ocorrencia para o ano 2016...
>> Baixando 2016/datatran2016_atual.csv
Buscando datasets de acidentes_ocorrencia para o ano 2015...
>> Baixando 2015/datatran2015.csv
Buscando datasets de acidentes_ocorrencia para o ano 2014...
>> Baixando 2014/datatran2014.csv
Buscando datasets de acidentes_ocorrencia para o ano 2013...
>> Baixando 2013/datatran2013.csv
Buscando datasets de acidentes_ocorrencia para o ano 2012...
>> Baixando 2012/datatran2012.csv
Buscando datasets de acidentes_ocorrencia para o ano 2011...
>> Baixando 2011/datatran2011.csv
Buscando datasets de acidentes_ocorrencia para o ano 2010...
>> Baixando 2010/datatran2010.csv
Buscando datasets de acidentes_ocorrencia pa

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


>>> Criando dataframe com ano 2015...
>>> Criando dataframe com ano 2014...
>>> Criando dataframe com ano 2013...
>>> Criando dataframe com ano 2012...


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


>>> Criando dataframe com ano 2011...
>>> Criando dataframe com ano 2010...
>>> Criando dataframe com ano 2009...
>>> Criando dataframe com ano 2008...
>>> Criando dataframe com ano 2007...


In [ ]:
accidents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36577 entries, 0 to 36576
Data columns (total 31 columns):
ano                       31313 non-null float64
br                        36572 non-null object
causa_acidente            36577 non-null object
classificacao_acidente    36577 non-null object
condicao_metereologica    36577 non-null object
data_inversa              36577 non-null object
delegacia                 3638 non-null object
dia_semana                36577 non-null object
fase_dia                  36577 non-null object
feridos                   36577 non-null int64
feridos_graves            36577 non-null int64
feridos_leves             36577 non-null int64
horario                   36577 non-null object
id                        36577 non-null float64
ignorados                 36577 non-null int64
ilesos                    36577 non-null int64
km                        36572 non-null object
latitude                  3638 non-null object
longitude                 3638 n

In [ ]:
accidents_df.tail()

,ano,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,feridos_graves,feridos_leves,horario,id,ignorados,ilesos,km,latitude,longitude,mortos,municipio,pessoas,regional,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos,infracoes
36572,2007.0,226,Outras,Com Vítimas Fatais,Ceu Claro,23/12/2007,NaN,Domingo,Anoitecer,1,0,1,17:30:00,561294.0,2,0,225,NaN,NaN,2,FLORANIA,5,NaN,Decrescente,Capotamento,Simples,Reta,RN,NaN,Rural,1,0
36573,2007.0,101,Outras,Com Vítimas Feridas,Ceu Claro,03/08/2007,NaN,Sexta,Anoitecer,1,1,0,18:55:00,586492.0,1,0,84.8,NaN,NaN,0,SAO GONCALO DO AMARANTE,2,NaN,Crescente,Atropelamento de pessoa,Dupla,Reta,RN,NaN,Urbano,1,0
36574,2007.0,304,Falta de atenção,Com Vítimas Feridas,Ceu Claro,24/06/2007,NaN,Domingo,Anoitecer,2,2,0,17:50:00,620680.0,0,1,281.5,NaN,NaN,0,MACAIBA,3,NaN,Decrescente,Colisão Transversal,Simples,Reta,RN,NaN,Rural,2,0
36575,2007.0,101,Outras,Com Vítimas Feridas,Ceu Claro,28/09/2007,NaN,Sexta,Plena noite,2,1,1,21:30:00,83384908.0,1,1,108.6,NaN,NaN,0,PARNAMIRIM,4,NaN,Crescente,Colisão lateral,Dupla,Cruzamento,RN,NaN,Urbano,2,0
36576,2007.0,101,Outras,Com Vítimas Feridas,Nublado,02/03/2007,NaN,Sexta,Pleno dia,1,0,1,17:00:00,909381.0,0,0,106,NaN,NaN,0,PARNAMIRIM,1,NaN,Crescente,Queda de motocicleta / bicicleta / veículo,Dupla,Cruzamento,RN,NaN,Urbano,1,0


## Cleaning the data

The first step is to normalize the format of the date.

In [ ]:
repl = lambda x: x.group(0)[-4:] + '-' + x.group(0)[-7:-5] + '-'+ x.group(0)[:2]
accidents_df['data'] = accidents_df.data_inversa.str.replace("[0-9]{2}/[0-9]{2}/[0-9]{4}",
                                                            repl, regex=True)

# Check out the result
print(accidents_df['data'].head())
print(accidents_df['data'].tail())

0    2019-01-01
1    2019-01-01
2    2019-01-01
3    2019-01-01
4    2019-01-02
Name: data, dtype: object
36572    2007-12-23
36573    2007-08-03
36574    2007-06-24
36575    2007-09-28
36576    2007-03-02
Name: data, dtype: object


In [ ]:
accidents_df.dropna(subset=['km', 'br'], inplace=True)

## Merging the datasets

In this section, we will aggregate the info about driving violations

In [ ]:
# Initialize the new column
accidents_df['infracoes'] = 0

In [ ]:
# Recover data
violations_df = pd.read_csv('violations_count.csv', sep=';', names=['data','km',
                                                                    'br',
                                                                    'contagem'])

# Check out the data
violations_df.head()

,data,km,br,contagem
0,2007-01-01,0,427.0,0
1,2007-01-01,1,427.0,0
2,2007-01-01,5,427.0,0
3,2007-01-01,9,427.0,0
4,2007-01-01,25,304.0,0


In [ ]:
violations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79410 entries, 0 to 79409
Data columns (total 4 columns):
data        79410 non-null object
km          79410 non-null int64
br          79410 non-null float64
contagem    79410 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 2.4+ MB


Now it's possible to update the number of driving violations based on the date of the violation and the point and road.

In [ ]:
def filter_violations(row):
  data_value = row['data']
  km_value = float(str(row['km']).replace(',', '.'))
  br_value = float(row['br'])
  # Recover violation ate the same point
  violation = violations_df.query("data == '{}' and km == {} and br == {}".format(
                                  data_value,km_value, br_value))
  
  return len(violation['contagem']) if violation['contagem'].sum() else 0


accidents_df['infracoes'] = accidents_df.apply(filter_violations, axis='columns')

In [ ]:
accidents_df.query('infracoes > 0')

,ano,br,causa_acidente,classificacao_acidente,condicao_metereologica,data_inversa,delegacia,dia_semana,fase_dia,feridos,feridos_graves,feridos_leves,horario,id,ignorados,ilesos,km,latitude,longitude,mortos,municipio,pessoas,regional,sentido_via,tipo_acidente,tipo_pista,tracado_via,uf,uop,uso_solo,veiculos,infracoes,data
780,NaN,304,Falta de Atenção à Condução,Com Vítimas Feridas,Céu Claro,2018-01-05,DEL15/1,sexta-feira,Pleno dia,1,0,1,08:15:00,101090.0,1,0,283,-5.86006,-35.3498,0,MACAIBA,2,SR-RN,Crescente,Colisão lateral,Simples,Reta,RN,UOP02/RN,Não,2,1,2018-01-05
797,NaN,101,Desobediência às normas de trânsito pelo condutor,Com Vítimas Feridas,Céu Claro,2018-01-11,DEL15/1,quinta-feira,Pleno dia,1,0,1,10:00:00,102631.0,2,0,106,-5.90584,-35.2603,0,PARNAMIRIM,3,SR-RN,Decrescente,Queda de ocupante de veículo,Dupla,Reta,RN,UOP01/RN,Não,2,1,2018-01-11
802,NaN,101,Desobediência às normas de trânsito pelo condutor,Com Vítimas Feridas,Céu Claro,2018-01-12,DEL15/2,sexta-feira,Pleno dia,3,1,2,15:00:00,102932.0,0,0,73,-5.70687,-35.2669,0,NATAL,3,SR-RN,Decrescente,Colisão transversal,Simples,Interseção de vias,RN,UOP01/RN,Não,2,1,2018-01-12
818,NaN,406,Falta de Atenção à Condução,Com Vítimas Feridas,Chuva,2018-01-15,DEL15/2,segunda-feira,Pleno dia,1,0,1,08:30:00,103745.0,0,0,169,-5.75935,-35.2972,0,SAO GONCALO DO AMARANTE,1,SR-RN,Crescente,Capotamento,Dupla,Não Informado,RN,UOP01/RN,Não,1,1,2018-01-15
819,NaN,405,Desobediência às normas de trânsito pelo condutor,Sem Vítimas,Céu Claro,2018-01-15,DEL15/4,segunda-feira,Pleno dia,0,0,0,14:05:00,103767.0,1,1,1,-5.19144,-37.3593,0,MOSSORO,2,SR-RN,Decrescente,Colisão lateral,Dupla,Reta,RN,UOP02/RN,Sim,2,1,2018-01-15
821,NaN,226,Falta de Atenção à Condução,Com Vítimas Feridas,Céu Claro,2018-01-16,DEL15/3,terça-feira,Pleno dia,1,0,1,11:30:00,103940.0,0,1,112,-6.2265,-36.0168,0,SANTA CRUZ,2,SR-RN,Decrescente,Colisão frontal,Simples,Reta,RN,UOP02/RN,Sim,2,1,2018-01-16
848,NaN,405,Não guardar distância de segurança,Com Vítimas Feridas,Sol,2018-01-26,DEL15/4,sexta-feira,Pleno dia,1,1,0,15:20:00,106317.0,0,1,2,-5.1925,-37.3692,0,MOSSORO,2,SR-RN,Crescente,Colisão traseira,Dupla,Interseção de vias,RN,UOP02/RN,Sim,2,1,2018-01-26
862,NaN,406,Falta de Atenção à Condução,Sem Vítimas,Céu Claro,2018-01-29,DEL15/2,segunda-feira,Pleno dia,0,0,0,11:10:00,107065.0,0,2,162,-5.7226,-35.365,0,CEARA-MIRIM,2,SR-RN,Decrescente,Colisão transversal,Simples,Não Informado,RN,UOP01/RN,Sim,2,1,2018-01-29
874,NaN,110,Falta de Atenção à Condução,Com Vítimas Feridas,Nublado,2018-02-01,DEL15/4,quinta-feira,Plena Noite,1,0,1,18:45:00,107920.0,0,1,45,-5.18804,-37.3441,0,MOSSORO,2,SR-RN,Crescente,Colisão transversal,Simples,Reta,RN,UOP01/RN,Sim,2,1,2018-02-01
892,NaN,101,Falta de Atenção à Condução,Com Vítimas Feridas,Céu Claro,2018-02-05,DEL15/1,segunda-feira,Pleno dia,2,0,2,14:00:00,108946.0,0,0,96,-5.84725,-35.2085,0,NATAL,2,SR-RN,Crescente,Colisão lateral,Dupla,Reta,RN,UOP01/RN,Sim,2,1,2018-02-05


In [ ]:
accidents_df.to_csv('accidents.csv', sep=';')